分数获取

In [5]:
import os
import re
from docx import Document
import pandas as pd
import numpy as np
from itertools import permutations


file_path="C:\\Users\\lenovo\\Desktop\\ICNALE GRA 2_1.xlsx"
sheet_index = 5
data = pd.read_excel(file_path, sheet_name=sheet_index)

if 'Sample Info.1' in data.columns and 'Unnamed: 32' in data.columns:
    filter_condition = data['Sample Info.1'].str.contains('Essay', na=False)
    # 筛选符合条件的行，并选择对应列
    filtered_data = data.loc[filter_condition, ['Sample Info.1', 'Unnamed: 32']]
    print(filtered_data)
else:
    print("列 Sample Info.1 或 Unnamed: 32 不存在，请检查数据列名")

    Sample Info.1 Unnamed: 32
1       Essay_001    62.15625
2       Essay_002   43.259375
3       Essay_003   74.690625
4       Essay_004   39.796875
5       Essay_005    43.38125
..            ...         ...
136     Essay_136    48.86875
137     Essay_137    57.99375
138     Essay_138     37.4875
139     Essay_139   63.871875
140     Essay_140   45.253125

[140 rows x 2 columns]


文本答案获取

In [6]:
def extract_number(filename):
    """从文件名中提取数字部分用于排序"""
    match = re.search(r'(\d{3})', filename)
    return int(match.group(1)) if match else -1  # 如果没有匹配到，则返回-1
folder_path = "C:\\Users\\lenovo\\Desktop\\ICNALE GRA2_1\\Rating Samples\\ICNALE_GRA_Original Essays"  # 替换为你的文件夹路径
all_data = []
files = [f for f in os.listdir(folder_path) if f.endswith('.docx') and not f.startswith('.')]
sorted_files = sorted(files, key=extract_number)

for filename in sorted_files:
    file_path = os.path.join(folder_path, filename)
    try:
        doc = Document(file_path)
        text = '\n'.join([para.text for para in doc.paragraphs])  # 合并所有段落为一个字符串
        all_data.append(text)
    except Exception as e:
        print(f"Failed to read {filename}: {e}")
print(all_data)

["In my opinion, it is necessary for college students to have part-time job once their school grade or other condition become steady. Why? First, college students are old enough to be financially dependent. They are no more kids, and they have the responsibility to lessen the pressure from their parent. They have to understand how hard is working, thus, they can save and never waste their money. Second, they are no more students in a few years, so they have to judge what kind of job suit them through taking part-time jobs. Take me as an example. As a male, although I have to do the military responsibility first, and after one year then I have to work. I am very afraid now. Many of my classmates are worried about finding no job, and I may face the same situation after one year. One of my classmates declined a chance to be a teacher in the kindergarten, because she found herself having no interest in education. And some of my classmates felt sad and worried after the interview of a job, 

分数与文本匹配-形成dataframe结构

In [17]:
scores_array = filtered_data['Unnamed: 32'].values  # 转化为NumPy数组
# 假设scores_df中的索引与texts列表中的顺序一致
combined_df = pd.DataFrame({
    'Score': scores_array,
    'Text': all_data
})

# 按照分数进行等间隔采样，确保分数分布均匀
# 取10个数据
indices = np.round(np.linspace(0, len(combined_df) - 1, 10)).astype(int)
selected_pairs = combined_df.iloc[indices]

import json
selected_texts = selected_pairs['Text'].tolist()
selected_scores=selected_pairs['Score'].tolist()
# 保存为JSON文件
with open('selected_texts.json', 'w', encoding='utf-8') as f:
    json.dump(selected_texts, f, ensure_ascii=False, indent=4)

print("文本已保存为 selected_texts.json")

with open('selected_scores.json', 'w', encoding='utf-8') as f:
    json.dump(selected_scores, f, ensure_ascii=False, indent=4)

print("文本已保存为 selected_scores.json")

0      In my opinion, it is necessary for college stu...
15     In college, students want to find something to...
31     A part-time job for college student ? Hmm.... ...
46     Nowadays, more and more college students think...
62     I agree that it is important for college stude...
77     Now I am studying at Mae Fah Luang University,...
93     In order to be an independent adult, we have t...
108    I think there are several reasons it is import...
124    As we all know, some college students have par...
139    I agree with it is important for college stude...
Name: Text, dtype: object


成对组合

In [14]:
import json
items = selected_pairs.values.tolist()
# 使用permutations生成所有可能的有序对
pairs = list(permutations(items, 2))
answers_array = []
for idx,pair in enumerate(pairs,1):
    temp_str=f'''answer 1:\n"{pair[0][1]}"\nanswer 2:\n"{pair[1][1]}"\n'''
    answers_array.append(temp_str)
with open('answer_pairs.json', 'w') as f:
    json.dump(answers_array, f)

answer 1:
"In my opinion, it is necessary for college students to have part-time job once their school grade or other condition become steady. Why? First, college students are old enough to be financially dependent. They are no more kids, and they have the responsibility to lessen the pressure from their parent. They have to understand how hard is working, thus, they can save and never waste their money. Second, they are no more students in a few years, so they have to judge what kind of job suit them through taking part-time jobs. Take me as an example. As a male, although I have to do the military responsibility first, and after one year then I have to work. I am very afraid now. Many of my classmates are worried about finding no job, and I may face the same situation after one year. One of my classmates declined a chance to be a teacher in the kindergarten, because she found herself having no interest in education. And some of my classmates felt sad and worried after the interview o

大模型输出处理函数

In [ ]:
import ast
def parse_json(model_output):
    if type(model_output) is dict:
        return model_output
    elif type(model_output) is not str:
        model_output = str(model_output)
    try:
        model_output = model_output.replace("\n", " ")
        model_output = re.search('({.+})', model_output).group(0)
        model_output = re.sub(r"(\w)'(\w|\s)", r"\1\\'\2", model_output)
        result = ast.literal_eval(model_output)
    except (SyntaxError, NameError, AttributeError):
        return "ERR_SYNTAX"
    return result

简单成对比较生成标准

In [ ]:
import json
from tqdm import tqdm

system_prompt_criteria_generating = '''
You are a professional educational evaluator who is responsible for evaluating student essays of 200-300 words on the importance of part-time work for college students.\
Now, you are asked to compare two student essays, decide which one is better, and explain your findings.\
Please base your evaluation on the following criteria and give 2-3 criteria to support your conclusion.\
Next I will give you an example, the final result should be returned in JSON format, strictly following the example format provided below.
<example>
given input:
answer 1:
"In my opinion, it is necessary for college students to have part-time job once their school grade or other condition become steady. Why? First, college students are old enough to be financially dependent. They are no more kids, and they have the responsibility to lessen the pressure from their parent. They have to understand how hard is working, thus, they can save and never waste their money. Second, they are no more students in a few years, so they have to judge what kind of job suit them through taking part-time jobs. Take me as an example. As a male, although I have to do the military responsibility first, and after one year then I have to work. I am very afraid now. Many of my classmates are worried about finding no job, and I may face the same situation after one year. One of my classmates declined a chance to be a teacher in the kindergarten, because she found herself having no interest in education. And some of my classmates felt sad and worried after the interview of a job, because they found themselves having no the skills or experiences that the boss wanted. So I think it's good to test college students' ability by having part-time job."
answer 2:
"In college, students want to find something to do to have the new experience and money. There are many ways to find something to do and the best one is part time job. It is good for students to do the part time job because it will make the students get more experiences, students will earn money, and they may have the opportunities to get the job in future. Firstly, it is good for students to do the part time jobs because students will get more experiences. When students have to do the jobs, they will face many situations from consumers and the boss of them. They will have more patients and learn more to live with others in societies. Secondly, students will earn more money while they are learning. Part time job makes money for students too. Student will know the value of money and how hard they find them. From this, they will know how to use money and they will have money if they have to use it. Finally, another reason why it is good for students to have a part time job is they might get more opportunities for their works in future. If students work, they will meet many people. In addition, if the agency finds that this student is good at working, they can remember your name and reserve this student for doing the job when this student graduates. All in all, the students might strongly believe that to do the part time job is necessary and important. Because part time job can give more experiences, make money, and provide more opportunities for student in the future."

You should output:
{
    "better_answer": "answer 1",
    "criteria":[
        "logic",
        "deep of content",
        "expression"
    ]
}
</example>

'''
def init_compare_answers_with_llm(system_prompt,pair_data):
    model_name = "/home/linshiyi/.cache/modelscope/hub/ZhipuAI/glm-4-9b-chat/"
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype="auto",
        device_map="auto",
        trust_remote_code=True
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    data=[]
    for user_prompt in tqdm(pair_data):
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            trust_remote_code=True
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        result = parse_json(response)
        print(result)
        data.append(result)

    with open('init_criteria.json', 'a+', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
init_compare_answers_with_llm(system_prompt_criteria_generating,answers_array)

读取初始标准

In [ ]:
with open('init_criteria.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
# Flatten the list of criteria
all_criteria = [criterion for item in data for criterion in item['criteria']]
model_name = "/home/linshiyi/.cache/modelscope/hub/ZhipuAI/glm-4-9b-chat/"
system_prompt_rank = (
    "You are a helpful and detail-oriented assistant. You will be provided with a list of criteria extracted from a dataset. "
    "Your task is to analyze these criteria and rank them based on the following principles:\n\n"
    "1. Repetition: Criteria that appear more frequently across the dataset are considered more important.\n"
    "2. Importance: Criteria that are generally recognized as fundamental for evaluation should have higher priority.\n\n"
    "Provide a sorted list of the criteria, starting with the most important. "
    "Next I will give you an example, the final result should be returned in JSON format, strictly following the example format provided below."
    '''<example>
    given input:
     The criteria to rank are:"depth of content","clarity of argument","organization","logic","depth of content","expression"
    You should output:
    {
        "Rank_criteria": [
            "depth of content",
            "logic",
            "clarity of argument"
            "organization"
            "expression"
        ]
    }
    </example>
    '''
)
def rank_criteria_with_llm(criteria_list, system_prompt, model_name, output_file):
    # Load the model and tokenizer
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype="auto",
        device_map="auto",
        trust_remote_code=True
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Prepare the user prompt by combining all criteria
    user_prompt = f"The criteria to rank are:\n{', '.join(criteria_list)}"

    # Create messages for the chat template
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        trust_remote_code=True
    )

    # Prepare input for the model
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # Generate response
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    # Decode response
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Parse response and save to file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(response)

    print("Ranking completed. Output saved to", output_file)

rank_criteria_with_llm(all_criteria, system_prompt_rank, model_name, output_file="ranked_criteria.json")

加载排序好的标准文件
并取Top-K（10）

In [ ]:
with open("ranked_criteria.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

# Flatten the list of criteria

ranked_criteria=data['Rank_criteria'][:10]

根据Top-K再次进行答案成对对比——在标准a下答案i与答案j谁好
形成k\*n\*n的矩阵

In [ ]:
def generate_comparison_prompt(criteria, answer):
    return (
        f"You are a helpful assistant. Compare two answers based on the following evaluation criterion: '{criteria}'.\n\n"
        f"{answer}\n\n"
        "Which answer better satisfies the criterion? Respond with one of the following options:\n"
        "'5' if Answer 1 is much better than Answer 2,\n"
        "'3' if Answer 1 is slightly better than Answer 2,\n"
        "'1' if both are equal,\n"
        "'1/3' if Answer 2 is slightly better than Answer 1,\n"
        "'1/5' if Answer 2 is much better than Answer 1."
    )
def compare_answers_with_llm(pair_data,top_criteria, model_name, output_file):
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype="auto",
        device_map="auto",
        trust_remote_code=True
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    n = 10
    k = len(top_criteria)
    A_kij = np.ones((k, n, n), dtype=float)  # 初始化 A_kij 矩阵
    for k_idx, criterion in enumerate(tqdm(top_criteria, desc="Processing criteria")):
        for i in range(n):
            temp = 0
            for j in range(n):
                if i == j:
                    continue
                else:
                    system_prompt = generate_comparison_prompt(criterion, pair_data[temp])
                    temp += 1

                # 准备输入
                messages = [
                    {"role": "system", "content": system_prompt},
                ]
                text = tokenizer.apply_chat_template(
                    messages,
                    tokenize=False,
                    add_generation_prompt=True,
                    trust_remote_code=True
                )
                model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

                # 调用大模型生成输出
                generated_ids = model.generate(
                    **model_inputs,
                    max_new_tokens=512,
                )
                generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
                # 解析输出
                try:
                    last_line = generated_text.splitlines()[-1]
                    result = int(last_line)
                    print(result)
                    A_kij[k_idx, i, j] = result
                    A_kij[k_idx, j, i] = 1/result  # 对称性
                except ValueError:
                    print(
                        f"Unexpected output for criterion '{criterion}' between answers {i} and {j}: {generated_text}")


    np.save(output_file, A_kij)
    print(f"Matrix saved to {output_file}")
compare_answers_with_llm(answers_array,ranked_criteria,model_name, output_file="evaluation_under_criteria.npy")

标准权重矩阵生成

In [ ]:

with open('ranked_criteria.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
rank_criteria = data["Rank_criteria"]

# 取前10个标准
top_10_criteria = rank_criteria[:10]

# 初始化评分矩阵
n = len(top_10_criteria)
cri_matrix = np.ones((n, n),dtype='float')  # 初始化为1，因为a=a时评分为1
# 根据规则填写矩阵
for i in range(n):
    for j in range(n):
        if i == j:
            continue  # 自己与自己比较，跳过

        if rank_criteria.index(top_10_criteria[i]) < rank_criteria.index(top_10_criteria[j]):
            cri_matrix[i, j] = 3  # a > b
            cri_matrix[j, i] = 1/3  # 对称的关系
        elif rank_criteria.index(top_10_criteria[i]) > rank_criteria.index(top_10_criteria[j]):
            cri_matrix[i, j] = 1/3  # a < b
            cri_matrix[j, i] = 3  # 对称的关系
        else:
            cri_matrix[i, j] = 1  # a == b
            cri_matrix[j, i] = 1  # 对称的关系

标准矩阵以及评分矩阵最大特征值对应特征向量求解


In [ ]:
## 归一化函数
def normalize_vector(v):
    total = np.sum(v)  # 计算向量的总和
    normalized_v = v / total  # 每个元素除以总和
    return normalized_v
## 特征向量求解函数
def sigma(matrix):
    eigenvalues, eigenvectors = np.linalg.eig(matrix)

    # 获取最大特征值对应的索引
    max_eigenvalue_index = np.argmax(eigenvalues)

    # 返回最大特征值对应的特征向量
    return eigenvectors[:, max_eigenvalue_index]


# 三维评分矩阵获取
matrix_3d = np.load('evaluation_under_criteria.npy')
m, n, _ = matrix_3d.shape  # m 是矩阵层数，n 是每个二维矩阵的维度

# 初始化结果矩阵，存储每个二维矩阵对应的最大特征向量
normal_scores_eigenvectors_matrix = np.zeros((m, n),dtype='float')  # 结果矩阵形状为 (m, n)

# 对每个二维矩阵计算最大特征值对应的特征向量并进行归一化
for i in range(m):
    normal_scores_eigenvectors_matrix[i, :] = normalize_vector(sigma(matrix_3d[i]))

cri_eigenvectors_matrix=np.real(sigma(cri_matrix))
normal_cri_eigenvectors_matrix=normalize_vector(cri_eigenvectors_matrix)

最终分数矩阵计算以及CI一致性指数计算

In [ ]:
final_scores=np.dot(normal_scores_eigenvectors_matrix.T,normal_cri_eigenvectors_matrix)
print(final_scores)

##一致性指数计算
def calculate_consistency_index(final_scores, real_score):
    n = len(final_scores)
    consistent_pairs = 0
    total_comparable_pairs = n * (n - 1) // 2

    for i in range(n):
        for j in range(i + 1, n):
            if (final_scores[i] > final_scores[j] and real_score[i] > real_score[j]) or \
                    (final_scores[i] < final_scores[j] and real_score[i] < real_score[j]):
                consistent_pairs += 1

    ci = consistent_pairs / total_comparable_pairs if total_comparable_pairs != 0 else 0
    return ci
CI_result=calculate_consistency_index(final_scores, scores_array)
print(CI_result)